

---

# Part 3: The Network and Transport Layers – Addressing, Routing, and Reliable Delivery

## Chapter 9: The Router and the Routing Table

In Chapter 6, we introduced the concept of the default gateway—the router that acts as the exit from a local network. In Chapter 7, we learned how to carve up address space into efficient subnets. In Chapter 8, we explored the vast world of IPv6. Now, it is time to examine the device that ties all these concepts together and makes internetworking possible: the **router**.

While switches operate at Layer 2 and use MAC addresses to forward frames within a local network, routers operate at Layer 3 and use IP addresses to forward packets **between** networks. A router is a specialized computer whose sole purpose is to examine incoming packets, consult its internal map (the routing table), and make intelligent decisions about the best path to forward each packet toward its final destination.

This chapter will dissect the router and its most critical data structure, the routing table. You will learn about directly connected networks, static routes, and the fundamentals of dynamic routing. You will understand the path selection process and the tools used to troubleshoot it. By the end of this chapter, you will have a solid understanding of how packets navigate the complex mesh of networks that form the Internet.

### 9.1 The Router: The Core L3 Device for Interconnecting Networks

At its simplest, a router is a device that connects multiple networks together. A typical home router connects your internal home network (e.g., `192.168.1.0/24`) to your ISP's network, which is connected to the rest of the Internet. In an enterprise, a router might connect a sales department VLAN to a engineering department VLAN, or connect a campus network to a WAN link.

A router has several key characteristics:

- **Multiple Network Interfaces:** A router has at least two network interfaces, each connected to a different IP network. Each interface is assigned an IP address that belongs to the network it is connected to. This IP address serves as the default gateway for devices on that network.

- **Operates at Layer 3:** Routers make forwarding decisions based on Layer 3 information—specifically, the destination IP address in the packet header. They strip off the incoming Layer 2 frame, examine the IP packet, and then create a brand new Layer 2 frame for the outgoing interface. This is why routers are able to connect networks that use different Layer 2 technologies (e.g., Ethernet on one side and a WAN serial link on the other).

- **Maintains a Routing Table:** The router's intelligence resides in its **routing table**. This is a data structure stored in memory that contains information about known networks and how to reach them. When a packet arrives, the router consults this table to determine the best next hop.

- **Does Not Forward Broadcasts:** Unlike switches, which flood Layer 2 broadcasts, routers do not forward Layer 3 broadcasts. They are designed to contain broadcast traffic within a single network segment. This is a fundamental reason why routers are used to create scalable, high-performance internetworks.

**Analogy: The Air Traffic Controller**
Think of a router as an air traffic controller at a major airport. Packets are like airplanes arriving from various destinations. The air traffic controller (router) does not care about the physical condition of the plane (the Layer 2 frame). Instead, they look at the flight plan—the destination (the IP address). They consult their radar and schedules (the routing table) to determine the best runway and departure gate (the outgoing interface) to send that plane on its way to its final destination, possibly via another airport (another router). The controller guides each plane independently, ensuring they all take the most efficient path.

### 9.2 Anatomy of a Routing Table

A router's routing table is a collection of entries, or **routes**, each of which tells the router how to reach a particular destination network. You can view the routing table on most routers and even on your own computer. On a Windows machine, you can type `route print` in the command prompt. On a macOS or Linux machine, you can type `netstat -rn`.

A typical routing table entry contains several key pieces of information:

- **Destination Network:** The network address (in CIDR notation) that this route describes. For example, `192.168.1.0/24` or `10.0.0.0/8`. This is the network we are trying to reach.

- **Next Hop (or Gateway):** The IP address of the next router to which the packet should be forwarded. This is the immediate neighbor that knows how to get the packet closer to its final destination. If the destination is directly connected, this field may show "on-link" or `0.0.0.0`.

- **Interface:** The specific outgoing interface on the router that should be used to reach the next hop or the destination network. For example, `GigabitEthernet0/0` or `eth0`.

- **Metric (or Cost):** A value that indicates the "distance" or "cost" of reaching the destination via this route. If there are multiple routes to the same destination, the router will prefer the one with the lowest metric. The metric can be based on hop count, bandwidth, delay, or other factors, depending on the routing protocol.

- **Administrative Distance (AD):** This is a value that indicates the "trustworthiness" of the source of the route. If a router learns about the same destination network from two different sources (e.g., from a static configuration and from a dynamic routing protocol like OSPF), it will use the Administrative Distance to decide which source to believe. The lower the AD, the more trustworthy the source.

Let's look at a simplified example of a routing table:

| Destination Network | Next Hop | Interface | Metric | Administrative Distance | Source |
| :--- | :--- | :--- | :--- | :--- | :--- |
| 192.168.1.0/24 | On-link | Gi0/0 | 0 | 0 | Directly Connected |
| 10.10.10.0/24 | 192.168.1.100 | Gi0/0 | 1 | 1 | Static |
| 0.0.0.0/0 (Default) | 203.0.113.1 | Gi0/1 | 20 | 110 | OSPF (Dynamic) |
| 172.16.0.0/16 | 192.168.1.102 | Gi0/0 | 3 | 120 | RIP (Dynamic) |

In this table, the router has learned about the `172.16.0.0/16` network from both a static route (AD 1) and RIP (AD 120). Because the static route has a lower AD, it would be considered more trustworthy and would be installed in the routing table as the active route. If the static route were to fail, the router would fall back to the RIP-learned route.

### 9.3 The Path Selection Process: Longest Prefix Match

When a router receives a packet, it examines the destination IP address. It then looks through its routing table to find the best matching route. The algorithm it uses is called the **longest prefix match**.

The principle is simple: among all the routes in the table that match the destination IP address, the router chooses the one with the **most specific (longest) subnet mask**.

Consider a router with the following three routes in its table:

1.  `0.0.0.0/0` (the default route)
2.  `192.168.0.0/16`
3.  `192.168.1.0/24`

Now, a packet arrives destined for `192.168.1.100`.

- Does it match the default route (`0.0.0.0/0`)? Yes, the default route matches everything.
- Does it match `192.168.0.0/16`? Yes, because the first 16 bits of the destination (`192.168.`) match the route.
- Does it match `192.168.1.0/24`? Yes, because the first 24 bits of the destination (`192.168.1.`) match the route.

All three routes match. Which one does the router choose? The longest prefix match rule says: choose the route with the longest subnet mask. A `/24` mask is longer (more specific) than a `/16`, and a `/16` is longer than a `/0`. Therefore, the router will forward the packet using the route to `192.168.1.0/24`.

This is a critical concept. It allows for **route summarization** (using less specific routes) to keep routing tables small, while also allowing for **exception routes** (using more specific routes) to direct traffic for a particular subnet along a different path.

### 9.4 Administrative Distance and Metrics: Choosing the Best Route

The longest prefix match tells the router which route is the most specific. But what if there are two different routes in the table to the exact same destination network, with the exact same prefix length? For example, a router might learn about `10.10.10.0/24` from both OSPF and EIGRP. Which one should it use?

This is where **Administrative Distance (AD)** and **Metrics** come into play. They operate at different stages of the route selection process.

**Administrative Distance (AD):**
AD is a value that ranks the **trustworthiness of the route source**. It is used when a router has multiple paths to the same destination learned from *different* routing protocols or sources. The route with the lowest AD is installed in the routing table.

Here are the default AD values for common route sources on Cisco routers (other vendors use similar, but not identical, values):

| Route Source | Default Administrative Distance |
| :--- | :--- |
| Directly Connected | 0 |
| Static Route | 1 |
| EIGRP (Summary Route) | 5 |
| External BGP | 20 |
| Internal EIGRP | 90 |
| OSPF | 110 |
| IS-IS | 115 |
| RIP | 120 |
| External EIGRP | 170 |
| Internal BGP | 200 |
| Unknown / Unusable | 255 |

A directly connected network (AD 0) is always the most trustworthy. A static route manually configured by an administrator (AD 1) is more trustworthy than a route learned dynamically from OSPF (AD 110). If a router learns about the same network from OSPF and RIP, it will choose the OSPF route because 110 is lower than 120.

**Metric:**
Metric is a value used to compare multiple paths to the same destination that were learned from the **same routing protocol**. For example, if OSPF learns of two different paths to `10.10.10.0/24`, it will calculate a metric for each path (based on link bandwidth, in OSPF's case) and choose the path with the **lowest metric** to insert into the routing table.

Different routing protocols use different metrics:
- **RIP:** Hop count (number of routers to cross).
- **OSPF:** Cost (based on link bandwidth; faster links have a lower cost).
- **EIGRP:** A composite metric based on bandwidth and delay (by default).

In summary:
1.  **Longest Prefix Match** selects the most specific route.
2.  If multiple sources provide the same most-specific route, **Administrative Distance** selects the most trustworthy source.
3.  If a single source provides multiple paths to the same destination, **Metric** selects the best path according to that protocol's rules.

### 9.5 Static Routes: Manual Configuration

A **static route** is a route that a network administrator manually configures on a router. It is a simple, hard-coded entry in the routing table.

**Example:** On a Cisco router, the command to add a static route might look like this:
```
ip route 10.10.20.0 255.255.255.0 192.168.1.100
```
This tells the router: "To reach the network `10.10.20.0/24`, send packets to the next-hop router at `192.168.1.100` via whatever interface is on that subnet."

**Pros of Static Routing:**
- **Simple and Predictable:** Easy to configure and understand in small networks.
- **Low Overhead:** No routing protocol traffic is generated; the router's CPU is not consumed by complex calculations.
- **Secure:** No routing information is shared with other routers, which can be a security benefit in some circumstances.

**Cons of Static Routing:**
- **Not Scalable:** In a network with dozens or hundreds of routers, manually configuring all the routes would be a nightmare.
- **No Automatic Adaptation:** If a link fails, a static route cannot automatically switch to a backup path. An administrator must manually reconfigure the route. This is the biggest disadvantage.

Static routes are best suited for small networks with simple, stable topologies, or for defining a specific path (like a default route) on the edge of a larger network.

### 9.6 Dynamic Routing Protocols (Introduction)

To solve the scalability and adaptability problems of static routing, we use **dynamic routing protocols**. These are protocols that allow routers to automatically learn about networks from their neighbors, share that information with other routers, and build a complete picture of the network topology.

When a network changes (a link goes down, a new network is added), routers running a dynamic routing protocol automatically detect the change, recalculate the best paths, and update each other's routing tables. This is called **convergence**. A good routing protocol converges quickly and without loops.

Dynamic routing protocols are classified based on where they operate and how they operate.

- **Interior Gateway Protocols (IGPs):** These protocols are used for routing **within** a single autonomous system (AS)—a network under a single administrative control, such as a company or a university. Examples include RIP, OSPF, and EIGRP.
- **Exterior Gateway Protocols (EGPs):** These protocols are used for routing **between** different autonomous systems. The only EGP in use today is **BGP (Border Gateway Protocol)** , the protocol that runs the entire Internet.

Within IGPs, there are two main types of algorithms used:

- **Distance Vector Protocols:** These protocols operate on the principle of "routing by rumor." A router knows only the direction (vector) and distance to a destination, based on information from its directly connected neighbors. It sends its entire routing table to its neighbors at regular intervals. RIP is a classic example. They are simple but can be slow to converge and prone to routing loops.

- **Link-State Protocols:** These protocols operate on the principle of "having a map." Every router builds a complete picture (a topological map) of the entire network by exchanging "link-state advertisements" (LSAs) with all other routers. Each router then independently runs an algorithm (the Shortest Path First or Dijkstra algorithm) to calculate the best path to every destination. OSPF and IS-IS are link-state protocols. They are more complex but converge faster and are less prone to loops.

We will explore these protocols in much greater detail in the next chapter.

### 9.7 The Internet Control Message Protocol (ICMP): Ping and Traceroute

Before we move on, we must introduce a crucial companion protocol to IP: the **Internet Control Message Protocol (ICMP)** . ICMP is used by network devices, including routers, to send error messages and operational information. It is the protocol that powers two of the most essential network troubleshooting tools: **ping** and **traceroute**.

ICMP messages are encapsulated directly within IP packets. They are not a transport layer protocol like TCP or UDP; they are considered an integral part of the Internet Layer.

**Ping (Packet Internet Groper):**
Ping uses two specific ICMP message types:
- **ICMP Echo Request (Type 8):** Sent by the source device.
- **ICMP Echo Reply (Type 0):** Sent back by the destination device if it is reachable.

When you type `ping 8.8.8.8`, your computer sends an ICMP Echo Request to that address. If the destination is reachable and configured to respond, it sends back an ICMP Echo Reply. Ping then reports the success and the round-trip time (RTT). This confirms basic IP connectivity and provides a measure of latency.

**Traceroute (tracert on Windows, traceroute on macOS/Linux):**
Traceroute is a more sophisticated tool that maps the path a packet takes from source to destination. It exploits the **Time-to-Live (TTL)** field in the IP header.

The TTL field is a counter that decrements by one every time a packet passes through a router. Its purpose is to prevent packets from looping forever. When a router receives a packet with a TTL of 1, it decrements it to 0 and discards the packet. It then sends an ICMP **Time Exceeded (Type 11, Code 0)** message back to the original source.

Traceroute works by sending a series of packets with incrementally increasing TTL values:
1.  It sends a packet with TTL = 1. The first router decrements it to 0, discards it, and sends back an ICMP Time Exceeded message. Traceroute now knows the IP address of the first-hop router.
2.  It sends a packet with TTL = 2. The first router decrements it to 1 and forwards it. The second router decrements it to 0, discards it, and sends back an ICMP Time Exceeded message. Traceroute now knows the second-hop router.
3.  This process continues until a packet finally reaches the destination. The destination, upon receiving a packet with a TTL of 1 (or higher), will not send a Time Exceeded message. Instead, it will send an ICMP Echo Reply (if the packet was an Echo Request) or a **Port Unreachable (Type 3, Code 3)** message if it was a UDP packet (which is what `traceroute` on Linux/macOS uses by default).

By collecting the source IP addresses of all the ICMP Time Exceeded messages and the final reply, traceroute reconstructs the entire path.

---

### Chapter 9: Hands-On Challenge

Let's explore routing on your own network and computer.

1.  **View Your Local Routing Table:**
    - Open your command line or terminal.
    - **Windows:** Type `route print` and press Enter.
    - **macOS/Linux:** Type `netstat -rn` and press Enter.
    - You will see a list of routes. Look for:
        - Your directly connected network (e.g., `192.168.1.0` with a mask of `255.255.255.0` and a Gateway of `On-link` or `0.0.0.0`).
        - Your default route (`0.0.0.0` with a mask of `0.0.0.0`). The Gateway column shows your default gateway's IP address.

2.  **Perform a Traceroute:**
    - **Windows:** Type `tracert 8.8.8.8`
    - **macOS/Linux:** Type `traceroute 8.8.8.8`
    - Observe the output. The first hop is your local router (default gateway). Subsequent hops are the routers your packets traverse as they leave your network and cross the Internet. You may see some hops that reply with `* * *` – this means that particular router is configured not to respond to traceroute probes, which is common for security reasons.

3.  **Observe ICMP in Wireshark:**
    - Open Wireshark and start a capture on your active interface.
    - In your command line, ping a remote server: `ping 8.8.8.8` (on Windows, it will ping continuously; press Ctrl+C to stop).
    - Stop the Wireshark capture.
    - In the filter bar, type `icmp` and press Enter.
    - You will see the ICMP Echo Request packets you sent and the ICMP Echo Reply packets you received. Examine their structure.

4.  **Simulate a Static Route (Conceptual):**
    - If you have access to a router (like a home router with advanced firmware, or a lab environment), you could try configuring a static route. For example, if you had a second router connected to your main router, you could add a static route on the main router telling it that the network behind the second router is reachable via the second router's IP address. This exercise is more advanced and requires lab equipment.

---

This chapter has introduced you to the brains of the network layer. You now understand that routers are not just simple forwarding devices; they are intelligent computers that build and maintain maps (routing tables) and make complex decisions (longest prefix match, AD, metric) about how to deliver packets.

In the next chapter, we will dive deep into the most common and important dynamic routing protocols used in enterprise networks. We will explore the differences between distance-vector and link-state protocols and get hands-on with **RIP, OSPF, and EIGRP**.